# Data Overview

## Context
Ookla is the global leader in network intelligence and connectivity insights. The company owns the world-renowned Speedtest platform which is the definitive way to test the speed and performance of your internet connection. Every day, over ten million unique tests are actively initiated by Ookla's users in the locations and at the times when their connectivity matters to them.

Since Speedtest's founding in 2006, an unparalleled total of more than 40 billion tests have been taken with the pform.ng 2020-01-01)

## Content

The datasets provide global fixed broadband and mobile (cellular) network performance metrics in zoom level 16 web mercator tiles (approximately 610.8 meters by 610.8 meters at the equator). **Download speed**, **upload speed**, and **latency** are collected via the Speedtest by Ookla applications for Android and iOS and averaged for each tile. Measurements are filtered to results containing GPS-quality location accuracy.

1. **Name**: Identifies the name of the location, server, or region where the data was collected (e.g., country, city).
2. **Number of Records**: The total number of data entries or measurements recorded for the location or server.
3. **Devices**: The number of devices involved in the test or contributing to the internet speed data for that location or server.
4. **Tests**: The total number of internet speed tests performed in the dataset for the given location or server.
5. **Avg. Avg U Kbps**: The average upload speed (in kilobits per second) from all the tests conducted at the given location or server.
6. **Avg. Avg D Kbps**: The average download speed (in kilobits per second) from all the tests conducted at the given location or server.
7. **Avg Lat Ms**: The average latency (in milliseconds) measured during the tests, indicating how quickly data is transmitted to and from the server.
8. **Avg. Pop2005**: Refers to the population estimate from 2005 for the given location or region.
9. **Rank Upload**: The rank of the location or server based on upload speeds, with 1 being the highest rank (fastest upload speeds).
10. **Rank Download**: The rank of the location or server based on download speeds, with 1 being the highest rank (fastest download speeds).

## Source
The performance datasets are available via AWS S3 at the s3://ookla-open-data bucket, where individual Parquet time series and compressed Shapefiles are organized by

- file format (shapefiles or parquet)
- service type (fixed or mobile)
- year (2020)
- quarter (for example, 1 corresponds to the Q1 period starting 2020-01-01)

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import os

In [2]:
from warnings import filterwarnings
filterwarnings(action="ignore")

# Load Data

In [3]:
# Simple Function to Aggregate Data to one DataFrame 
def aggregate(csv_files):
    df = pd.DataFrame(columns=['Year','Quarter','type','Number of Records', 'Devices', 'Tests', 'Avg. Avg U Kbps',
       'Avg. Avg D Kbps', 'Avg Lat Ms', 'Avg. Pop2005', 'Rank Upload',
       'Rank Download', 'Rank Latency'])
    for csv_file in csv_files:
        current_df = pd.read_csv(f"../data/{csv_file}")
        current_df = current_df[current_df['Name'] == 'Algeria']
        current_df.drop("Name",axis=1,inplace=True)

        # Extract Infos from the name of the file and Add it to current_df
        infos = csv_file.split(".csv")[0].split("/")[-1]
        current_df['Year'] = infos.split("year")[-1][1:5]
        current_df['Quarter'] = infos.split("quarter")[-1][1:]
        current_df['type'] = infos.split("year")[0].replace("_","")

        if 'Number of Record' in current_df.columns:
            current_df.drop('Number of Record', axis=1, inplace=True)
                    
        df = pd.concat([df,current_df],axis=0)
    df = df.reset_index().drop("index",axis=1)
    
    return df

In [4]:
# Loaded Data
data = aggregate(os.listdir("../data"))
data.head()

,Year,Quarter,type,Number of Records,Devices,Tests,Avg. Avg U Kbps,Avg. Avg D Kbps,Avg Lat Ms,Avg. Pop2005,Rank Upload,Rank Download,Rank Latency
0,2020,01,fixed,"11,913","39,923","170,080","1,368","3,492",61,"32,854,159",230,227,60
1,2020,02,fixed,"12,220","39,516","194,803",991,"3,130",55,"32,854,159",234,228,73
2,2020,03,fixed,"12,760","45,257","215,847","1,083","3,934",47,"32,854,159",232,225,70
3,2020,04,fixed,"12,163","40,447","180,785","1,395","4,512",49,"32,854,159",232,225,64
4,2021,01,fixed,"14,053","54,011","261,733","1,235","4,939",46,"32,854,159",233,224,69


In [5]:
# Rename columns
data.rename(columns={
    'Number of Records':'NumberRecords',
    'Avg. Avg U Kbps': 'Avg_Avg_U_Kbps',
    'Avg. Avg D Kbps': 'Avg_Avg_D_Kbps',
    'Avg Lat Ms': 'Avg_Lat_Ms',
    'Avg. Pop2005': 'Avg_Pop2005',
    'Rank Upload': 'Rank_Upload',
    'Rank Download': 'Rank_Download',
    'Rank Latency': 'Rank_Latency',
},inplace=True)

In [6]:
# Sanitize float values (replace NaN, Infinity, -Infinity)
def sanitize_float(value):
    if isinstance(value, float):
        if value != value or value in [float('inf'), float('-inf')]:  # Check NaN or Inf
            return ""
    return value if value is not None else ""

for col in data.columns:
    data[col] = data[col].apply(sanitize_float)

In [7]:
# Columns Data Types
data.dtypes

Year              object
Quarter           object
type              object
NumberRecords     object
Devices           object
Tests             object
Avg_Avg_U_Kbps    object
Avg_Avg_D_Kbps    object
Avg_Lat_Ms        object
Avg_Pop2005       object
Rank_Upload        int64
Rank_Download      int64
Rank_Latency       int64
dtype: object

In [8]:
# Convert Each Column to each appropriate type
numerical_cols = ["Year","Quarter","Devices","Tests","Avg_Avg_U_Kbps","Avg_Avg_D_Kbps","Avg_Lat_Ms","Avg_Pop2005"]
for col in numerical_cols:
    data[col] = data[col].apply(lambda X: str(X).replace(",","").replace(",","")).astype(int)

# Basic infos about the dataset

In [9]:
# Memory Usage in megabytes
print(f"Dataset's memory size: {data.memory_usage().sum()/1e3:.3f} KB")

Dataset's memory size: 1.716 KB


In [10]:
# Dataset's Shape & Size
print(f"Number of Columns(Features/Attributes): {data.shape[1]}")
print(f"Number of Rows(observations): {data.shape[0]}")

print(f"Size: {data.size}")

Number of Columns(Features/Attributes): 13
Number of Rows(observations): 22
Size: 286


In [11]:
# Columns - Number of Non Nulls and Data types
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Year            22 non-null     int32 
 1   Quarter         22 non-null     int32 
 2   type            22 non-null     object
 3   NumberRecords   22 non-null     object
 4   Devices         22 non-null     int32 
 5   Tests           22 non-null     int32 
 6   Avg_Avg_U_Kbps  22 non-null     int32 
 7   Avg_Avg_D_Kbps  22 non-null     int32 
 8   Avg_Lat_Ms      22 non-null     int32 
 9   Avg_Pop2005     22 non-null     int32 
 10  Rank_Upload     22 non-null     int64 
 11  Rank_Download   22 non-null     int64 
 12  Rank_Latency    22 non-null     int64 
dtypes: int32(8), int64(3), object(2)
memory usage: 1.7+ KB
None


# Upload Raw Data to the API

In [12]:
upload_url = "http://127.0.0.1:8000/upload/raw"
response = requests.post(upload_url,json={"content": data.to_dict(orient="records")})
response.status_code

200

In [13]:
response.json()

{'detail': 'Upload Data Successfully'}